In [1]:
library(tensorflow)
library(keras)
#library(tfdatasets)
library(tidyverse)
library(reticulate)
library(dplyr)
#library(sp)
#library(jsonlite)
#library(data.table)
#library(imager)
  SIZE <- 256L
  SPLIT<- 1024L
  thr<- 0.4

 gpu_devices =  tf$config$experimental$list_physical_devices("GPU")[[1]]
 tf$config$experimental$set_memory_growth(gpu_devices, T)

ff<- list.files("../input/reqssss",full.names=T )   
py_install(ff)

 py<- function(x)py_run_string(x)


rasterio<- import("rasterio", convert=F)
np<- import("numpy", convert = F)
#npx<- import("numpy", convert = T)

win<- rasterio$windows
logging<- rasterio$logging
log<- rasterio$logging$getLogger()
log$setLevel(logging$ERROR)


TEST<- file.path("../input/hubmap-kidney-segmentation/test")
BASE<-file.path("../input/hubmap-kidney-segmentation/")
TRAIN<- file.path("../input/hubmap-kidney-segmentation/train" )
MODEL<- file.path("../input/effb3-models")





bce_dice_loss <- function(y_true, y_pred) {
    result <- loss_binary_crossentropy(y_true, y_pred) +
        (1 - dice(y_true, y_pred))
    return(result)
} 

dice <- custom_metric("dice", function(y_true, y_pred, smooth = 1e-5) {
  y_true_f <- k_flatten(y_true)
  y_pred_f <- k_flatten(y_pred)
  intersection <- k_sum(y_true_f * y_pred_f)
  (2 * intersection + smooth) / (k_sum(y_true_f) + k_sum(y_pred_f) + smooth)
})
cross_loss <- function(y_true, y_pred) {
    # y_true<-k_cast(y_true, "float32")
    # y_pred<-k_cast(y_pred,"float32")
    result <- loss_binary_crossentropy(y_true, y_pred) + loss_logcosh(y_true, y_pred)* (1 - dice(y_true, y_pred)) #loss_logcosh(y_true, y_pred)* (1 - dice(y_true, y_pred))     # 
    return(result)
} 

fload<-function(x){ 
    mm<- load_model_hdf5(file.path(x), custom_objects = c("cross_loss"= cross_loss, "dice"=dice), compile = F)
    mm
}     
modelList<-list.files(MODEL,pattern = ".hdf5",full.names = T)

models<- map(modelList,fload)
gc()

test<-read_csv(file.path(BASE,"sample_submission.csv"))
#dataset<- read_csv(file.path(BASE,"HuBMAP-20-dataset_information.csv"))
baseid<- test$id

b<- tibble(baseid=baseid, size = list.files(TEST, full.names = T,pattern = "*.tiff") %>% file.size/1e9, k = 1: length(baseid))

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.1.1     ✔ dplyr   1.0.5
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



Using virtual environment '/usr/local/share/.virtualenvs/r-reticulate' ...


None

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2105310,112.5,4224960,225.7,4224960,225.7
Vcells,3603339,27.5,8388608,64.0,5856708,44.7



── Column specification ────────────────────────────────────────────────────────
cols(
  id = col_character(),
  predicted = col_logical()
)




In [2]:

idx2rle <- function(idx) {
  if (length(idx) == 0) return("")
  
  # Make sure  values are sorted
  idx <- sort(idx)
  
  # Array of starting positions and run lengths
  starts <- c()
  runs <- c()
  
  # Loop
  starts <- c(starts, idx[1])
  run <- 1
  for (i in 2:length(idx)) { #run
    if (idx[i-1] + 1 == idx[i]) {
      run <- run + 1
    } else { #stop
      runs <- c(runs, run)
      starts <- c(starts, idx[i])
      run <- 1
    }
  }
  
  # Last run length needs special handling
  runs <- c(runs, run)
  
  # Sanity check
  stopifnot(length(starts) == length(runs))
  
  # zip starts and runs as space delimited list
  return(paste0(mapply(list, paste(starts, runs)), collapse = " "))
  
}


make_nn<-function(height,width){
  cc<-SPLIT/2
  ncolsi<-c( seq(0,width -2*cc, cc),width-2*cc)
  nrowsi<-c( seq(0,height-2*cc,  cc), height - 2*cc)
  nn<-expand.grid(nrowsi, ncolsi)
  names(nn)<- c("y","x")
  #nn$id<- 1:nrow(nn)
  nn$x <- as.integer( nn$x)
  nn$y <- as.integer(nn$y)
  #single<- big [(SPLIT/2+1):(SPLIT/2 + SPLIT) ,(SPLIT/2+1):(SPLIT/2 + SPLIT) ]
  return(nn) 
}

to_batch_predict<-function(i,j,im=im,channellast){

  if(channellast){
    imx<-im[(i+1L):(i+SPLIT),(j+1L):(j+SPLIT),] %>%
    tf$image$convert_image_dtype(.,tf$float32) 
  }else
  {  imx<-im[,(i+1L):(i+SPLIT),(j+1L):(j+SPLIT)] %>%
    k_permute_dimensions(c(2L,3L,1L)) %>%
    tf$image$convert_image_dtype(.,tf$float32)
  }

 imx<- imx  %>%
  tf$image$resize( size = shape(SIZE,SIZE)) 
 imx
}

modes <- function(d){
    i <- which(diff(sign(diff(d$y))) < 0) + 1
    data.frame(x = d$x[i], y = d$y[i])
}
remove_bw<-function(h,w,layers){
  #lets try to filter based on the density information
  ### now, applied to all
  xx <- read_from_layers(h, w,layers) %>% as.array %>% density( n = 64) %>% modes(.) %>% nrow(.)
  return(xx > 10)
}



read_from_layers<- function(h,w,layers){
    ww<-win$Window(w,h,SPLIT,SPLIT)
    if(length(layers) ==1){
      nlayers<- map(1:3,function(i){layers[[1]]$read(i,window=ww)})
       out<-tf$stack(nlayers,2L)
    }else{
      nlayers<- map(layers,function(x){x$read( 1L ,window=ww) } )
      out<-tf$stack(nlayers,2L)
     }
     out<- out %>%  tf$image$convert_image_dtype(.,tf$float32) %>%
      tf$image$resize(., size = shape(SIZE,SIZE))
      return(out)
}
HuBMAP<-function(tiff_path){
    dataset = rasterio$open(tiff_path, num_threads="all_cpus")
    self_layers = list()
    if (dataset$count != 3){
      subdatasets = dataset$subdatasets
      if (length(subdatasets) > 0){
        for( i in 0: (length(subdatasets) -1) ){
        self_layers<- c(self_layers,(
          rasterio$open(subdatasets[i], num_threads="all_cpus")))
          }
        }
      }else{
        self_layers<- c(self_layers,(rasterio$open(tiff_path, num_threads="all_cpus")))
      }
    
    layers<- self_layers
    height <- dataset$height %>% py_to_r
    width<- dataset$width %>% py_to_r
    nnc<-make_nn(height,width)
    #ibw<- map2(nnc[,1],nnc[,2],remove_bw) %>% unlist
    ibw<-c()
    for( i in 1:nrow(nnc)){
       ibw <- c(ibw,remove_bw(nnc[i,1],nnc[i,2],layers) )
     } 
    gc()
    nnc<-nnc[!ibw,]
    
    chunk2 <- function(x,n) split(x, cut(seq_along(x), n, labels = FALSE))
    nn<-ceiling(nrow(nnc)/50) 
    nlist<-chunk2(1:nrow(nnc),nn)
return(list(nlist=nlist,layers=layers, nnc=nnc, height=height, width=width))
}

In [3]:
sub<-function(k){

on.exit(keras::backend()$clear_session())
 gc()
tiff_path<-file.path(TEST,paste0(baseid[k],".tiff") )

c(nlist,layers,nnc,height,width) %<-%   HuBMAP(tiff_path)

gen_preds<-function(ll){
  img<- map2(nnc[ll,1],nnc[ll,2],read_from_layers,layers)  %>%map(., function(x)tf$expand_dims(x,axis=0L)) %>%tf$concat(.,axis=0L) 
  #img<- map2(nnc[ll,1],nnc[ll,2],fa,channellast) %>% map(.,function(x)tf$expand_dims(x,axis=0L))%>% tf$concat(.,axis=0L)
  img<-  models %>% map(.,function(x)predict(x,img,batch_size=8)) %>%  tf$math$add_n(.)   %>%
    tf$math$divide(.,length(models)) %>%  tf$image$resize( size = shape(SPLIT,SPLIT) )   
  gc()
  return(img)
}      
gen_idx<-function(ll){
  preds<-gen_preds(ll)
    fidx<-function(i){
      nrowi<-nnc[i,1] + SPLIT/4
      ncoli<-nnc[i,2] + SPLIT/4
      big<- preds[which(ll==i),,,1] %>% as.array
      pred<- big[(SPLIT/4+1):(SPLIT/4 + SPLIT/2) ,(SPLIT/4+1):(SPLIT/4 + SPLIT/2 ) ]
      predx<- pred > thr  
      pred[predx]<-1L;pred[!predx]<-0L
       #if(sum(pred) ==0)return(NULL)
      coord <- which(predx == 1L, arr.ind = T) %>% as.data.frame
      idx <-
         with(coord, {
          (col + ncoli - 1L) * height + row + nrowi
        }) %>% as.integer #convert to original size
    idx
  }
  out<- ll %>% map(.,fidx) %>% unlist 
  gc()
  return(out)
}
                        
res<-     nlist %>% map(.,gen_idx) %>% unlist %>% unique %>% sort %>%  idx2rle()
gc()
return(res)
}

In [4]:
# library(imager)
# i=  324
# xn<-nnc[i,]
# img<- fa(xn[, 1], xn[, 2],channellast) %>% tf$expand_dims(axis=0L)
# img %>% tf$squeeze(.) %>% as.array %>% as.raster %>% plot
# 
# pred<- xs_single(i)
# pred %>% as.raster %>% plot
# summary(pred %>% as.numeric )

In [5]:
tsglobal<-proc.time()


In [6]:

sub_catch = possibly(.f = sub, otherwise = "10 2 20 3")

#predicted<-map(1:length(baseid),sub_catch)
predicted<-list()
for( i in 1:length(baseid)){
    predicted<- c( predicted,sub(i))
    gc()
    cat( baseid[i],"\n")
}
#predicted<- list(sub(1))
hand<-tibble(id=baseid ,predicted=predicted %>% unlist)
submission<-read_csv(file.path(BASE,"sample_submission.csv"))
submission[,2]<- hand[,2] 
#submission[1,]<-hand






2ec3f1bb9 
3589adb90 
d488c759a 
aa05346ff 
57512b7f1 



── Column specification ────────────────────────────────────────────────────────
cols(
  id = col_character(),
  predicted = col_logical()
)




In [7]:
tt<-proc.time() -tsglobal
cat(paste0("elapsed : ", tt[[3]]/60 , " minutes"))


elapsed : 56.4201666666667 minutes

In [8]:
write_csv(submission,"submission.csv")
cat("write_csv\n")

write_csv


In [9]:
file.size("submission.csv")


[1] 5666017